In [1]:
import pandas as pd
df = pd.read_parquet("/home/jiaruil5/social_reasoning_rl/data/tomi/train.parquet")

In [2]:
df

,index,story,question,answer,data_source,prompt,ability,reward_model,extra_info
0,0,Mia entered the lounge. Chloe arrived at the l...,Where does Mia think that Chloe searches for t...,crate,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'crate', 'story': ...","{'index': 0, 'split': 'train'}"
1,1,Hunter crept into the cellar. Nathan leapt int...,Where will Nathan look for the sweater?,bathtub,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'bathtub', 'story'...","{'index': 1, 'split': 'train'}"
2,2,Sophia stepped into the bedroom. The strawberr...,Where is the pumpkin really?,cupboard,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'cupboard', 'story...","{'index': 2, 'split': 'train'}"
3,3,Ava bounded into the TV_room. Owen walked into...,Where will Ava look for the sweet_potato?,drawer,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'drawer', 'story':...","{'index': 3, 'split': 'train'}"
4,4,Owen approached the porch. Hannah visited the ...,Where will Logan look for the scarf?,bottle,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'bottle', 'story':...","{'index': 4, 'split': 'train'}"
...,...,...,...,...,...,...,...,...,...
5999,5999,Oliver went to the kitchen. Mia bounded into t...,Where does Oliver think that Mia searches for ...,bucket,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'bucket', 'story':...","{'index': 5999, 'split': 'train'}"
6000,6000,Mason shuffled into the attic. James entered t...,Where is the gloves really?,crate,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'crate', 'story': ...","{'index': 6000, 'split': 'train'}"
6001,6001,Mia sidled into the playroom. Owen went to the...,Where will Owen look for the corn?,crate,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'crate', 'story': ...","{'index': 6001, 'split': 'train'}"
6002,6002,Jackson approached the crawlspace. Logan went ...,Where was the lettuce at the beginning?,crate,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'crate', 'story': ...","{'index': 6002, 'split': 'train'}"


In [9]:
df.iloc[0]['prompt']

array([{'content': "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.\nUser: Assume that the characters can perceive every scene in their location but not scenes occurring elsewhere. If something is being moved, that means it is not in its original location anymore. You should majorly focus on where the object has been moved to, and answer the question with the most detailed position possible e.g., the object is in A and A is in B, then you should answer 'A'. Here is the story: Mia entered the lounge. Chloe arrived at the lounge. Inside the container is a persimmon. The container is set in the lounge. Mia is in love with the strawberry. Mia shoved the persimmon into the crate. The crate is inside the lounge. Chloe ran from the lounge. Mia quit the lounge. Chloe entered the study. And here is the question: Where does Mia think tha

A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.\nUser: Assume that the characters can perceive every scene in their location but not scenes occurring elsewhere. If something is being moved, that means it is not in its original location anymore. You should majorly focus on where the object has been moved to, and answer the question with the most detailed position possible e.g., the object is in A and A is in B, then you should answer 'A'. Here is the story: Mia entered the lounge. Chloe arrived at the lounge. Inside the container is a persimmon. The container is set in the lounge. Mia is in love with the strawberry. Mia shoved the persimmon into the crate. The crate is inside the lounge. Chloe ran from the lounge. Mia quit the lounge. Chloe entered the study. And here is the question: Where does Mia think that Chloe searches for the persimmon?\nPlease show your reasoning in <think> </think> tags and provide shortest possible final answer in <answer> </answer> tags.\nAssistant: Let me solve this step by step.\n<think>

In [13]:
import numpy as np
from transformers import AutoTokenizer

# Load the tokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


def count_tokens(prompt_entry):
    if isinstance(prompt_entry, np.ndarray) and len(prompt_entry) > 0:
        prompt_list = prompt_entry.tolist()  # Convert NumPy array to list
        if (
            isinstance(prompt_list, list)
            and isinstance(prompt_list[0], dict)
            and "content" in prompt_list[0]
        ):
            text = prompt_list[0]["content"]  # Extract the 'content' text
            tokenized_output = tokenizer(text, return_tensors="pt")
            return tokenized_output["input_ids"].shape[1]  # Get token count
    return None  # Return None if input format is incorrect


# Apply token counting to DataFrame
df["num_tokens"] = df["prompt"].apply(count_tokens)

In [14]:
df

,index,story,question,answer,data_source,prompt,ability,reward_model,extra_info,num_tokens
0,0,Mia entered the lounge. Chloe arrived at the l...,Where does Mia think that Chloe searches for t...,crate,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'crate', 'story': ...","{'index': 0, 'split': 'train'}",250
1,1,Hunter crept into the cellar. Nathan leapt int...,Where will Nathan look for the sweater?,bathtub,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'bathtub', 'story'...","{'index': 1, 'split': 'train'}",239
2,2,Sophia stepped into the bedroom. The strawberr...,Where is the pumpkin really?,cupboard,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'cupboard', 'story...","{'index': 2, 'split': 'train'}",251
3,3,Ava bounded into the TV_room. Owen walked into...,Where will Ava look for the sweet_potato?,drawer,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'drawer', 'story':...","{'index': 3, 'split': 'train'}",252
4,4,Owen approached the porch. Hannah visited the ...,Where will Logan look for the scarf?,bottle,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'bottle', 'story':...","{'index': 4, 'split': 'train'}",251
...,...,...,...,...,...,...,...,...,...,...
5999,5999,Oliver went to the kitchen. Mia bounded into t...,Where does Oliver think that Mia searches for ...,bucket,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'bucket', 'story':...","{'index': 5999, 'split': 'train'}",261
6000,6000,Mason shuffled into the attic. James entered t...,Where is the gloves really?,crate,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'crate', 'story': ...","{'index': 6000, 'split': 'train'}",242
6001,6001,Mia sidled into the playroom. Owen went to the...,Where will Owen look for the corn?,crate,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'crate', 'story': ...","{'index': 6001, 'split': 'train'}",276
6002,6002,Jackson approached the crawlspace. Logan went ...,Where was the lettuce at the beginning?,crate,tomi,[{'content': 'A conversation between User and ...,reasoning,"{'ground_truth': {'answer': 'crate', 'story': ...","{'index': 6002, 'split': 'train'}",246


In [15]:
df['num_tokens'].max()

302